In [9]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd
from time import sleep
from random import randint
from IPython.display import clear_output 

In [29]:
def get_jobs(keyword, num_jobs):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="./chromedriver", options=options)
    driver.set_window_size(1120, 1000)
    
    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=N&locId=1&locKeyword=United%20States&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=25&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.generalKeyword=%22it%22&sc.locationSeoString=us&locId=1&locT=N&sc.pageNumber=5&radius=25'
    driver.get(url)
    
    #jobs = []
    jobs = 0
    start_time = time.time()
    requests = 0
    df = pd.DataFrame(columns=["Title","Location","Company","Description"])
    
    while jobs < num_jobs:  #If true, should be still looking for new jobs.
        
        # Pause the loop
        sleep(randint(8,15))
    
        # Monitor requests
        requests += 1
        elapsed_time = time.time() - start_time
        print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_class_name("modal_closeIcon-svg").click()  #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(jobs) + "/" + str(num_jobs)))
            clear_output(wait = True)
            #if len(jobs) >= num_jobs:
            if jobs >= num_jobs:
                break

            job_button.click() 
            time.sleep(1)
            collected_successfully = False
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            
            
            #add job to df
            df = df.append({'Title':job_title,'Location':location,"Company":company_name,"Description":job_description}, ignore_index=True)
            jobs +=1
            

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            #print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, jobs))
            break

    #return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.
    return df


In [30]:
df = get_jobs("Project Manager", 100)
df.to_csv("glassdoor_com-jobs-10.csv",index=False)
#with open('Glassdoor-jobs.csv','a',newline='') as f:
#    df.to_csv(f, header=False, index=False)
#f.close()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".selected"}
  (Session info: chrome=78.0.3904.108)


In [32]:
df.to_csv("glassdoor_com-jobs-10.csv",index=False)